# Rat 05 forward model with linear elasticity

In [5]:
%matplotlib inline
from fenics import *
import numpy as np
import scipy
import matplotlib.pyplot as plt
import os
import le

import scipy.io
from scipy.interpolate import RegularGridInterpolator

class InterpolatedParameter(Expression):
    def __init__(self,X,Y,param_image,**kwargs):
        self.X=X #a numpy array giving the X-spacing of the image
        self.Y=Y #same for Y
        self.param_image=param_image #The image of measured material property
    
    def eval_cell(self,values,x,cell):
        interp_handle=RegularGridInterpolator((self.X,self.Y,self.Z),self.param_image)
        values[0]=interp_handle(x)


adc=InterpolatedParameter(range(0,param_image.shape[0]),\
                          range(0,param_image.shape[1]),\
                          range(0,param_image.shape[2]),\
                          param_image,\ #a 3d image of aparent diffusivity
                          degree=0)

adc=interpolate(adc,V) #use the fenics interpolate function to apply the interpolation to the DOFs​


# Create mesh and define function spaces
mesh = Mesh("./rat-data/rat3gmsh.xml")
cf = MeshFunction("size_t",mesh,"./rat-data/rat3gmsh_physical_region.xml")
V = VectorFunctionSpace(mesh, 'P', 1)
W = FunctionSpace(mesh, 'P',1) # Function space for tumor cells

# Define initial fraction of tumor cells
# phi_0 = Expression('exp(-a*pow(x[0], 2) - a*pow(x[1], 2))', 
#                  degree=2, a=5)
# phi_n = interpolate(phi_0, W)
# plt.figure()
# plot(phi_n,title="Initial tumor cell concentration")

# Variables
mu = 1.0
lam = 1.0
nu = lam/(2*(lam+mu))
beta = .001
omega = 2*mu*beta+3*lam*beta

# Define boundary condition
def boundary(x, on_boundary):
    return on_boundary
bc = DirichletBC(V, Constant((0.0,0.0)), boundary)

T = Expression((('0','0'),('0','0')),degree=1)
n = FacetNormal(mesh)

# Compare with alternative formulation
u = le.varprob1(V, W, T, n, bc, phi_n, mu, lam, beta)
vm = le.vm1(u,mu,lam,beta, phi_n, W)

# Plot solution
plt.figure()
plot(u, title='Displacement', mode='displacement')

# Plot stress
plt.figure()
plot(vm, title='Stress intensity')

# Compute magnitude of displacement
plt.figure()
u_magnitude = sqrt(dot(u, u))
u_magnitude = project(u_magnitude, W)
plot(u_magnitude, title='Displacement magnitude')

# Save solution to file in VTK format
os.system('rm ./output/elasticity/*')
ufile = XDMFFile('./output/elasticity/disp.xdmf')
u.rename("disp","")
ufile.write(u,0)
del ufile

NameError: name 'phi_n' is not defined

In [37]:
import scipy.io
tumor = scipy.io.loadmat("./rat-data/rat05/rat05tumor.mat")
tumor['rat3tbound']

array([[39, 26],
       [39, 25],
       [39, 24],
       [39, 23],
       [39, 22],
       [39, 21],
       [40, 20],
       [41, 20],
       [42, 20],
       [43, 20],
       [44, 20],
       [45, 20],
       [46, 21],
       [47, 22],
       [48, 23],
       [49, 24],
       [49, 25],
       [49, 26],
       [50, 28],
       [49, 29],
       [48, 30],
       [46, 31],
       [45, 31],
       [44, 31],
       [42, 30],
       [41, 30],
       [40, 28],
       [39, 26]], dtype=uint8)